### **Libraries**

In [59]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

### **Load Dataset**

Data is stored as an Excel file. So, we use `Pandas` to load data.

In [60]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [79]:
train_path = '../../data/train.csv'
test_path = '../../data/test.csv'

df = load_data(train_path)
df_test = load_data(test_path)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb3 in position 10: invalid start byte

### **Remove Extra Columns**

In this project we don't need the `id` , `name` , `host_id` , `host_name` columns, because it does not bring any value for model.

In [62]:
df.drop(columns=['id', 'name', 'host_id', 'host_name' , 'latitude' , 'longitude'], inplace=True)
df_test.drop(columns=['id', 'name', 'host_id', 'host_name', 'latitude' , 'longitude'], inplace=True)

### **Columns Typecasting**

First of all, we can change name of the columns for having better understaing on them.

In [63]:
def change_column_name(df):
  new_column_names = {
      'neighbourhood_group': 'Location',
      'neighbourhood': 'Area',
      'room_type': 'Type',
      'price': 'Price',
      'minimum_nights': 'MinDayNights',
      'number_of_reviews': 'CountReview',
      'last_review': 'LastDateReview',
      'reviews_per_month': 'AvgReview',
      'calculated_host_listings_count': 'TotalHostListings',
      'availability_365': 'DayAvailability'
  }
  df = df.rename(columns=new_column_names)
  return df

df = change_column_name(df)
df_test = change_column_name(df_test)

print(df.columns)

Index(['Location', 'Area', 'Type', 'Price', 'MinDayNights', 'CountReview',
       'LastDateReview', 'AvgReview', 'TotalHostListings', 'DayAvailability'],
      dtype='object')


Here we should make sure the data type of each column.

In [64]:
print(df.dtypes)

Location              object
Area                  object
Type                  object
Price                  int64
MinDayNights           int64
CountReview            int64
LastDateReview        object
AvgReview            float64
TotalHostListings      int64
DayAvailability        int64
dtype: object


There is no issue in terms of data type and they are showing correct format.

### **Handle Missing Values**

Here, we should check whether data has any missing value. I create a for loop to iterate among all columns to realize whether they have null values or not. I’m looking for the number of null values in every single column as well as the percentage of null values.

In [65]:
for col in df.columns:
    number_null = df.loc[: , col].isnull().sum()
    perc_null = (number_null / df.shape[0]) * 100
    print('{} - {} - %{}'.format(col, number_null, round(perc_null,3)))

Location - 0 - %0.0
Area - 0 - %0.0
Type - 0 - %0.0
Price - 0 - %0.0
MinDayNights - 0 - %0.0
CountReview - 0 - %0.0
LastDateReview - 5633 - %20.574
AvgReview - 5633 - %20.574
TotalHostListings - 0 - %0.0
DayAvailability - 0 - %0.0


The result shows we must have a different approach to handling null values:

**Categorical**:
* less than 5%, I drop the rows.
* between 5% and 30%, I impute with mode. `LastDateReview`
* More than 30%, create a new label as Unknown.

In [66]:
df['LastDateReview'] = df['LastDateReview'].fillna(df['LastDateReview'].mode()[0])

**Numerical:**
* between 0% and 10%, I impute with mean or median.
* between 10% to 60%, i impute with KNN. `AvgReview`
* More than 60%, I drop the rows.

However, the best way is consulting with expert domain.

In [67]:
imputer = KNNImputer(n_neighbors=5)
df[['AvgReview']] = imputer.fit_transform(df[['AvgReview']])

### **Handle Duplicate Rows**

Now we should handle duplicate rows. Since all values might be same, we just we need to check whether there are two rows that all values in all columns are the same or not.

In [68]:
duplicate_rows = df.duplicated()

if duplicate_rows.any():
    print("The DataFrame has duplicate rows.")
else:
    print("The DataFrame does not have duplicate rows.")

The DataFrame has duplicate rows.


Now, we remove the duplicate rows.

In [69]:
df = df.drop_duplicates()

### **Numerical Sanity Check**

First of all, I declare all number variables. The number variables are: `Price` , `MinDayNights` , `CountReview` , `AvgReview` , `TotalHostListings` , `DayAvailability`

In [70]:
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

I must make sure about the data type of the number variable. Just because the column shows numbers, it doesn't mean that they are numbers.Thus, with regular expression I should clean them.

In [71]:
def check_non_numeric(df, column_name):
    non_numeric_values = df[column_name][~df[column_name].apply(lambda x: isinstance(x, (int, float)))].unique()

    if len(non_numeric_values) > 0:
        return True, non_numeric_values.tolist()
    else:
        return "There is no issue with"

for i in numeric_columns:
  print(check_non_numeric(df, i) , i)

There is no issue with Price
There is no issue with MinDayNights
There is no issue with CountReview
There is no issue with AvgReview
There is no issue with TotalHostListings
There is no issue with DayAvailability


The result shows all data are in correct and logical range.

### **Categorical Sanity Check**

First of all, I declare all categorical variables. These variables are: `Location`, `Area`, `Type`

In [72]:
cat_columns = df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

Then, we must make sure about the possible categories for each of them

In [73]:
def Cat_Checker(CatIndex):
    values = np.sort(df[cat_columns[CatIndex]].unique())
    return cat_columns[CatIndex] , values

In [74]:
Cat_Checker(0) #Location

('Location',
 array(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
       dtype=object))

In [75]:
Cat_Checker(1) #Area

('Area',
 array(['Allerton', 'Arden Heights', 'Arrochar', 'Arverne', 'Astoria',
        'Bath Beach', 'Battery Park City', 'Bay Ridge', 'Bay Terrace',
        'Baychester', 'Bayside', 'Bayswater', 'Bedford-Stuyvesant',
        'Belle Harbor', 'Bellerose', 'Belmont', 'Bensonhurst',
        'Bergen Beach', 'Boerum Hill', 'Borough Park', 'Breezy Point',
        'Briarwood', 'Brighton Beach', 'Bronxdale', 'Brooklyn Heights',
        'Brownsville', "Bull's Head", 'Bushwick', 'Cambria Heights',
        'Canarsie', 'Carroll Gardens', 'Castle Hill', 'Castleton Corners',
        'Chelsea', 'Chinatown', 'City Island', 'Civic Center',
        'Claremont Village', 'Clason Point', 'Clifton', 'Clinton Hill',
        'Co-op City', 'Cobble Hill', 'College Point', 'Columbia St',
        'Concord', 'Concourse', 'Concourse Village', 'Coney Island',
        'Corona', 'Crown Heights', 'Cypress Hills', 'DUMBO',
        'Ditmars Steinway', 'Dongan Hills', 'Douglaston',
        'Downtown Brooklyn', 'Dyker Hei

In [76]:
Cat_Checker(2) #Type

('Type',
 array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object))

### **CHECKPOINT**

In [77]:
%store df_test
%store df
%store numeric_columns
%store cat_columns

Stored 'df_test' (DataFrame)
Stored 'df' (DataFrame)
Stored 'numeric_columns' (list)
Stored 'cat_columns' (list)
